In [52]:
import pandas as pd
import numpy as np
from decimal import Decimal
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
df = pd.read_csv('/content/drive/MyDrive/rentology/dataset_rentology.csv')
df.head()

,Unnamed: 0,Location,Area,Bed,Bath,Price
0,0,"Block H, Bashundhara R-A, Dhaka","1,600 sqft",3,3,20 Thousand
1,1,"Farmgate, Tejgaon, Dhaka",900 sqft,2,2,20 Thousand
2,2,"Block B, Nobodoy Housing Society, Mohammadpur,...","1,250 sqft",3,3,18 Thousand
3,3,"Gulshan 1, Gulshan, Dhaka","2,200 sqft",3,4,75 Thousand
4,4,"Baridhara, Dhaka","2,200 sqft",3,3,75 Thousand


In [55]:
df.shape

(28800, 6)

In [56]:
#removing the unnamed column
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.head()

,Location,Area,Bed,Bath,Price
0,"Block H, Bashundhara R-A, Dhaka","1,600 sqft",3,3,20 Thousand
1,"Farmgate, Tejgaon, Dhaka",900 sqft,2,2,20 Thousand
2,"Block B, Nobodoy Housing Society, Mohammadpur,...","1,250 sqft",3,3,18 Thousand
3,"Gulshan 1, Gulshan, Dhaka","2,200 sqft",3,4,75 Thousand
4,"Baridhara, Dhaka","2,200 sqft",3,3,75 Thousand


In [57]:
df.isnull().sum()

Location    0
Area        0
Bed         0
Bath        0
Price       0
dtype: int64

In [58]:
#square feet er ", "gone
df['Area']=df['Area'].str.replace(',','')
df.head()

,Location,Area,Bed,Bath,Price
0,"Block H, Bashundhara R-A, Dhaka",1600 sqft,3,3,20 Thousand
1,"Farmgate, Tejgaon, Dhaka",900 sqft,2,2,20 Thousand
2,"Block B, Nobodoy Housing Society, Mohammadpur,...",1250 sqft,3,3,18 Thousand
3,"Gulshan 1, Gulshan, Dhaka",2200 sqft,3,4,75 Thousand
4,"Baridhara, Dhaka",2200 sqft,3,3,75 Thousand


In [59]:
# #removing the sqft from the area and making them int values
# df['Area'] = df['Area'].apply(lambda x: int(x.split(' ')[0]))
df['Area'] = df['Area'].astype(str).apply(lambda x: int(x.split(' ')[0]))
df.head()

,Location,Area,Bed,Bath,Price
0,"Block H, Bashundhara R-A, Dhaka",1600,3,3,20 Thousand
1,"Farmgate, Tejgaon, Dhaka",900,2,2,20 Thousand
2,"Block B, Nobodoy Housing Society, Mohammadpur,...",1250,3,3,18 Thousand
3,"Gulshan 1, Gulshan, Dhaka",2200,3,4,75 Thousand
4,"Baridhara, Dhaka",2200,3,3,75 Thousand


In [60]:
df.Bed.unique()


array([3, 2, 4, 1, 5, 6])

In [61]:
df.Bath.unique()


array([3, 2, 4, 5, 1, 6, 8])

In [62]:
df[df.Bed>5]


,Location,Area,Bed,Bath,Price
304,"Gulshan 2, Gulshan, Dhaka",4500,6,6,2.8 Lakh
24464,"Tejturi Bazar, Tejgaon, Dhaka",2450,6,6,60 Thousand


In [63]:
df[df.Bath>6] #8 baths!! noice but why?????


,Location,Area,Bed,Bath,Price
533,"Gulshan 2, Gulshan, Dhaka",4200,4,8,3 Lakh


In [64]:
df1 = df.copy()


In [65]:
#I wanted the make the price Thousand/Lakh values to a float values
def price_float(x):
    y = x.split(' ')[1]
    if y == "Thousand":
        return float(x.split(' ')[0]) * 1000
    else:
        return float(x.split(' ')[0]) * 100000

In [66]:
df1.Price = df1.Price.apply(price_float)


In [67]:
df1.loc[304]


Location    Gulshan 2, Gulshan, Dhaka
Area                             4500
Bed                                 6
Bath                                6
Price                        280000.0
Name: 304, dtype: object

In [68]:
df1['price_per_sqft'] = df1['Price']/df1['Area']
df1.head()


,Location,Area,Bed,Bath,Price,price_per_sqft
0,"Block H, Bashundhara R-A, Dhaka",1600,3,3,20000.0,12.500000
1,"Farmgate, Tejgaon, Dhaka",900,2,2,20000.0,22.222222
2,"Block B, Nobodoy Housing Society, Mohammadpur,...",1250,3,3,18000.0,14.400000
3,"Gulshan 1, Gulshan, Dhaka",2200,3,4,75000.0,34.090909
4,"Baridhara, Dhaka",2200,3,3,75000.0,34.090909


In [69]:
len(df1['Location'].unique())


730

In [70]:
#removing the comma in the Area column
df1['Location']=df1['Location'].str.replace(',','')
df1.head()

,Location,Area,Bed,Bath,Price,price_per_sqft
0,Block H Bashundhara R-A Dhaka,1600,3,3,20000.0,12.500000
1,Farmgate Tejgaon Dhaka,900,2,2,20000.0,22.222222
2,Block B Nobodoy Housing Society Mohammadpur Dhaka,1250,3,3,18000.0,14.400000
3,Gulshan 1 Gulshan Dhaka,2200,3,4,75000.0,34.090909
4,Baridhara Dhaka,2200,3,3,75000.0,34.090909


In [71]:
#Finding the locations with most number of houses
location_count = df1['Location'].value_counts(ascending=False)
location_count.head(30)


Location
Mohammadpur Dhaka                                               757
Mirpur Dhaka                                                    556
Block D Section 12 Mirpur Dhaka                                 417
Dhanmondi Dhaka                                                 414
Block E Section 12 Mirpur Dhaka                                 411
Sector 10 Uttara Dhaka                                          357
Paikpara Ahmed Nagar Mirpur Dhaka                               352
Kallyanpur Mirpur Dhaka                                         337
Section 12 Mirpur Dhaka                                         311
Block B Section 12 Mirpur Dhaka                                 307
Joar Sahara Dhaka                                               305
Block C Section 12 Mirpur Dhaka                                 294
West Shewrapara Mirpur Dhaka                                    292
Shyamoli Dhaka                                                  285
PC Culture Housing Mohammadpur Dhaka   

In [72]:
#Keeping the locations with less houses together
location_count_under_10 = location_count[location_count <= 10]

In [73]:
#now leveling them as 'other'
df1.Location = df1.Location.apply(lambda x: 'other' if x in location_count_under_10 else x)

In [74]:
#now we have less unique values which will incress our accuracy
len(df1['Location'].unique())

418

In [75]:
#now founding the unrealistic Area to Bed ratio according to my Civil engineer friend
df1[df1.Area/df1.Bed<300].head(10)

,Location,Area,Bed,Bath,Price,price_per_sqft
39,other,800,3,2,15000.0,18.750000
86,Block C Section 13 Mirpur Dhaka,745,3,2,15000.0,20.134228
135,other,800,3,2,15000.0,18.750000
186,Nayatola Maghbazar Dhaka,550,2,2,13000.0,23.636364
191,Baitul Aman Housing Society Adabor Dhaka,550,2,1,8500.0,15.454545
193,Taltola Khilgaon Dhaka,550,2,1,11000.0,20.000000
195,South Monipur Mirpur Dhaka,850,3,2,13000.0,15.294118
205,other,800,3,2,15000.0,18.750000
252,Block C Kaderabad Housing Society Bosila Moham...,750,3,2,16000.0,21.333333
260,Faydabad Dakshin Khan Dhaka,800,3,2,8500.0,10.625000


In [76]:
df1.shape

(28800, 6)

In [77]:
#removing them from our df
df2 = df1[~(df1.Area/df1.Bed<300)]
df2.shape

(27876, 6)

In [78]:
#Lets see the description of the 'price_per_sqft'
df2.price_per_sqft.describe()

count    27876.000000
mean        18.946916
std          6.575892
min          6.500000
25%         15.384615
50%         18.000000
75%         20.952381
max        228.571429
Name: price_per_sqft, dtype: float64

In [79]:
def drop_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('Location'):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        reduced_df = subdf[(subdf.price_per_sqft>(m-st)) & (subdf.price_per_sqft<=(m+st))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out
df3 = drop_outliers(df2)
df3.shape

(19913, 6)

In [80]:
df3[df3.Bath>df3.Bed+1]

,Location,Area,Bed,Bath,Price,price_per_sqft
747,Avenue 6 Mirpur DOHS Mirpur Dhaka,2175,3,5,42000.0,19.310345
748,Avenue 6 Mirpur DOHS Mirpur Dhaka,2175,3,5,42000.0,19.310345
1227,Banani DOHS Dhaka,3000,3,5,130000.0,43.333333
1271,Banani Dhaka,2410,3,5,80000.0,33.195021
1688,Block A Bashundhara R-A Dhaka,2100,3,5,60000.0,28.571429
1689,Block A Bashundhara R-A Dhaka,2100,3,5,60000.0,28.571429
3967,Block D Bashundhara R-A Dhaka,3000,3,5,55000.0,18.333333
4038,Block D Bashundhara R-A Dhaka,1700,2,4,30000.0,17.647059
4112,Block D Bashundhara R-A Dhaka,2200,3,5,30000.0,13.636364
6097,Block J Baridhara Dhaka,2200,3,5,50000.0,22.727273


In [81]:
dummies = pd.get_dummies(df3.Location)
dummies.head()

,1st Colony Mirpur Dhaka,2nd Colony Mirpur Dhaka,3rd Colony Mirpur Dhaka,Aainusbag Dakshin Khan Dhaka,Abdullahbag Uttar Badda Badda Dhaka,Abdur Rafique Housing Society Shekhertek Mohammadpur Dhaka,Adabor Dhaka,Adarsha Para Uttar Khan Dhaka,Agargaon Dhaka,Ahmed Nagar Mirpur Dhaka,...,West Kazipara Mirpur Dhaka,West Mollartek Dakshin Khan Dhaka,West Monipur Mirpur Dhaka,West Nakhalpara Tejgaon Dhaka,West Rampura Rampura Dhaka,West Shewrapara Mirpur Dhaka,West Vatara Vatara Badda Dhaka,Zafrabad Hazaribag Dhaka,aziz Moholla Mohammadpur Dhaka,other
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [82]:
df4 = pd.concat([df3,dummies.drop('other',axis='columns')],axis='columns')
df4.head()

,Location,Area,Bed,Bath,Price,price_per_sqft,1st Colony Mirpur Dhaka,2nd Colony Mirpur Dhaka,3rd Colony Mirpur Dhaka,Aainusbag Dakshin Khan Dhaka,...,West Kalachandpur Kalachandpur Dhaka,West Kazipara Mirpur Dhaka,West Mollartek Dakshin Khan Dhaka,West Monipur Mirpur Dhaka,West Nakhalpara Tejgaon Dhaka,West Rampura Rampura Dhaka,West Shewrapara Mirpur Dhaka,West Vatara Vatara Badda Dhaka,Zafrabad Hazaribag Dhaka,aziz Moholla Mohammadpur Dhaka
0,1st Colony Mirpur Dhaka,800,2,2,13500.0,16.875000,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1st Colony Mirpur Dhaka,700,2,2,12000.0,17.142857,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1st Colony Mirpur Dhaka,1100,3,3,20000.0,18.181818,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1st Colony Mirpur Dhaka,1100,3,3,20000.0,18.181818,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1st Colony Mirpur Dhaka,1020,3,2,15000.0,14.705882,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [83]:
df5 = df4.drop('Location',axis='columns')
df5.head()

,Area,Bed,Bath,Price,price_per_sqft,1st Colony Mirpur Dhaka,2nd Colony Mirpur Dhaka,3rd Colony Mirpur Dhaka,Aainusbag Dakshin Khan Dhaka,Abdullahbag Uttar Badda Badda Dhaka,...,West Kalachandpur Kalachandpur Dhaka,West Kazipara Mirpur Dhaka,West Mollartek Dakshin Khan Dhaka,West Monipur Mirpur Dhaka,West Nakhalpara Tejgaon Dhaka,West Rampura Rampura Dhaka,West Shewrapara Mirpur Dhaka,West Vatara Vatara Badda Dhaka,Zafrabad Hazaribag Dhaka,aziz Moholla Mohammadpur Dhaka
0,800,2,2,13500.0,16.875000,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,700,2,2,12000.0,17.142857,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1100,3,3,20000.0,18.181818,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1100,3,3,20000.0,18.181818,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1020,3,2,15000.0,14.705882,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [84]:
X = df5.drop(['Price','price_per_sqft'],axis='columns') #Had to drop 'price_per_sqft' because it was confusing the model and giving random negative predictions
X.head()

,Area,Bed,Bath,1st Colony Mirpur Dhaka,2nd Colony Mirpur Dhaka,3rd Colony Mirpur Dhaka,Aainusbag Dakshin Khan Dhaka,Abdullahbag Uttar Badda Badda Dhaka,Abdur Rafique Housing Society Shekhertek Mohammadpur Dhaka,Adabor Dhaka,...,West Kalachandpur Kalachandpur Dhaka,West Kazipara Mirpur Dhaka,West Mollartek Dakshin Khan Dhaka,West Monipur Mirpur Dhaka,West Nakhalpara Tejgaon Dhaka,West Rampura Rampura Dhaka,West Shewrapara Mirpur Dhaka,West Vatara Vatara Badda Dhaka,Zafrabad Hazaribag Dhaka,aziz Moholla Mohammadpur Dhaka
0,800,2,2,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,700,2,2,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1100,3,3,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1100,3,3,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1020,3,2,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [85]:
y = df5.Price
y.head()

0    13500.0
1    12000.0
2    20000.0
3    20000.0
4    15000.0
Name: Price, dtype: float64

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=15)

In [87]:
from sklearn.linear_model import Ridge
rdg_clf = Ridge(alpha=1.0)
rdg_clf.fit(X_train,y_train)
p = rdg_clf.score(X_test,y_test)
print("%.2f" % (p*100))
cv = ShuffleSplit(n_splits=5, test_size=0.25, random_state=10)

cross_val_score(Ridge(), X, y, cv=cv)

91.54


array([0.90683718, 0.90336858, 0.89706706, 0.91393935, 0.89846565])

In [88]:
def predict_price(Location,Area,Bed,Bath):
    index = np.where(X.columns == Location)[0][0]

    x = np.zeros(len(X.columns))
    x[0] = Area
    x[1] = Bed
    x[2] = Bath
    if index >= 0:
        x[index] = 1

    return rdg_clf.predict([x])[0] #Ridge Regression got the best accuracy

In [89]:
#My family used to live in this area in a 1100 sqft appertment with 3 beds and 3 baths. And the rent was 16500 Taka
predict_price('Matikata Cantonment Dhaka',1250, 3, 3)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(


17928.24219175511

In [90]:
predict_price('Mirpur Dhaka',1250, 3, 3)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(


20815.46440534888

In [91]:
predict_price('Adabor Dhaka',1300, 2, 2)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(


30445.870258171995

In [92]:
import pickle

In [93]:
filename = 'trained_model_dhaka2.sav'
pickle.dump(rdg_clf, open(filename, 'wb'))

In [94]:
# loading the saved model
loaded_model = pickle.load(open('trained_model_dhaka2.sav', 'rb'))

In [95]:
print(X)

       Area  Bed  Bath  1st Colony Mirpur Dhaka  2nd Colony Mirpur Dhaka  \
0       800    2     2                     True                    False   
1       700    2     2                     True                    False   
2      1100    3     3                     True                    False   
3      1100    3     3                     True                    False   
4      1020    3     2                     True                    False   
...     ...  ...   ...                      ...                      ...   
19908   800    2     2                    False                    False   
19909   900    2     2                    False                    False   
19910   740    2     2                    False                    False   
19911   780    2     2                    False                    False   
19912   740    2     2                    False                    False   

       3rd Colony Mirpur Dhaka  Aainusbag Dakshin Khan Dhaka  \
0                      

In [96]:
def predict_price(Location,Area,Bed,Bath):
    index = np.where(X.columns == Location)[0][0]

    x = np.zeros(len(X.columns))
    x[0] = Area
    x[1] = Bed
    x[2] = Bath
    if index >= 0:
        x[index] = 1

    return loaded_model.predict([x])[0]

In [97]:
predict_price('Adabor Dhaka',1300, 2, 2)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(


30445.870258171995

In [99]:
import pickle
pickle.dump(loaded_model, open('model_dhaka.pkl', 'wb'))